# Embedding  documento TXT en Chroma de forma persistente

https://github.com/hwchase17/chroma-langchain/blob/master/persistent-qa.ipynb

Un ejemplo del uso de Chroma DB y LangChain para responder preguntas sobre documentos, 
con una base de datos persistente localmente. 
Puede almacenar incrustaciones y documentos y luego utilizarlos nuevamente.

In [1]:
pip install -qU  langchain

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -qU  langchain-openai

Note: you may need to restart the kernel to use updated packages.


===========================================================================================================

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
openai_api_key = os.environ.get('OPENAI_API_KEY')

In [4]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

===========================================================================================================

## Cargar y procesar documentos

Cargue documentos para responder preguntas. Si desea hacer esto sobre sus documentos, esta es la sección que debe reemplazar.

A continuación dividimos los documentos en pequeños fragmentos. Esto es para que podamos encontrar los fragmentos más relevantes para una consulta y pasar solo esos al LLM.

In [5]:
# Load and process the text
loader = TextLoader('./data/DocumentacionMempool.txt')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

## Inicializar PeristedChromaDB

Cree incrustaciones para cada fragmento e insértelas en la base de datos de vectores Chroma. 

El argumento persist_directory le dice a ChromaDB dónde almacenar la base de datos cuando persiste.

In [6]:
# Embed and store the texts
# Al proporcionar un persist_directory se almacenarán las incrustaciones en el disco.
persist_directory = './Chroma/db_mempool'

embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)
vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding, 
                                 persist_directory=persist_directory
                                )

===========================================================================================================

## Conservar la base de datos

En un cuaderno, debemos llamar a **persist()** para asegurarnos de que las incrustaciones se escriban en el disco. Esto no es necesario en un script: la base de datos persistirá automáticamente cuando se destruya el objeto del cliente.

In [7]:
vectordb.persist()
vectordb = None